
<h1>Primera Entrega - Proyecto de Procesamiento de datos a gran escala</h1>

<h2>Fecha: 10/04/2024</h2>


<h4>Programas</h4>
<h5>Ciencia de datos e Ingeniería de Sistemas</h5>
<h4>Integrantes</h4>
<p>Santiago Botero Pacheco</p>
<p>Santiago Rueda Pineda</p>
<p>Santiago Avilés Tibocha</p>
<p>Brayan Steven Carrillo Mora</p>
<p style = 'text-align:center;'>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Javeriana.svg/1200px-Javeriana.svg.png" height="200" width="300">
</p>

<h3>Importando librerias y obteniendo el contenido de la pagina</h3>

In [0]:
import requests
from bs4 import BeautifulSoup #Libreria utilizada para realizar Web Scrapping

# URL de la página web
url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoodpop.htm'

# Realizar la solicitud GET para obtener el contenido HTML de la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML utilizando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar la tabla por su etiqueta <table> y cualquier identificador o clase si es necesario
    tabla = soup.find('table')
    #Se inicializan las listas donde se guardaran los datos obtenidos al obtener los datos de la pagina.
    loc = [] #Localidades de la ciudad de Nueva York
    bar = [] #Barrios de la ciudad de Nueva York
    hom = [] #Cantidad de hombres en Nueva York
    muj = [] #Cantidad de mujeres en Nueva York
    tot = [] #Total de habitantes en Nueva York
    if tabla:
        # Si se encuentra la tabla, puedes iterar sobre las filas y columnas para extraer la información
        i = 0 #Variable inicializada para saltarse la primera fila
        for fila in tabla.find_all('tr'): #Se buscan todas las filas 'rows' presentes en la tabla
            if i>0: #Establecemos condición para saltarnos el titulo de la tabla, es decir la primera fila
                localidades = fila.find_all('th', attrs={'headers': 'b'}) #Una vez analizada la pagina web de forma manual, se encontro que las localidades estan caracterizadas por el header 'b', por lo cual se busca especificamente los elementos <th> con este header en especifico.
                barrios = fila.find_all('th', attrs={'headers': 'c'})  #Una vez analizada la pagina web de forma manual, se encontro que los barrios estan caracterizadas por el header 'c', por lo cual se busca especificamente los elementos <th> con este header en especifico.
                columnas = fila.find_all('td') #Se buscan todas las columnas presentes en la fila, las cuales corresponden a los registros de la tabla
                if localidades: #Condición para verificar que se hayan obtenido localidades a través de Web Scrapping
                    for localidad in localidades: #Se recorren las localidades
                        if localidad.text.strip() != '': #Debido a que la tabla, muestra solamente una localidad para el primer barrio, y luego muestra un espacio vacio, haciendo alusión a que pertenecen a la ultima localidad mencionada, se debe verificar si el dato obtenido es un string vacio, en caso de no serlo, se agrega la localidad a la lista.
                            loc.append(localidad.text.strip())
                        else:
                            loc.append(loc[-1]) #En caso de ser un string vacio, se agrega el ultimo elemento presente en la lista de localidades.
                if barrios: #Condición para verificar que se hayan obtenido barrios a través de Web Scrapping
                    for barrio in barrios: #Se recorren los barrios
                        bar.append(barrio.text.strip())  #Se agregan los barrios al listado de barrios.
                if columnas:  # Verificar que la fila contenga datos
                    i = 0 #Se incializa variable para identificar si el valor de la columna corresponde a cantidad de hombres, mujeres o al total.
                    for columna in columnas:
                        if i == 0: #De forma manual, se identifico que la primera columna corresponde a la cantidad de hombres en los barrios.
                            hom.append(columna.text.strip())
                        if i == 1: #De forma manual, se identifico que la segunda columna corresponde a la cantidad de mujeres en los barrios.
                            muj.append(columna.text.strip())
                        if i == 2: #De forma manual, se identifico que la tercera columna corresponde a la cantidad total de personas en los barrios.
                            tot.append(columna.text.strip())
                        i = i + 1
            i = i+1
    else:
        print('No se encontró la tabla en la página web.')
else:
    print('La solicitud GET no fue exitosa. Código de estado:', response.status_code)

<h3>Formando un dataframe de Pandas a partir de la información obtenida a través de WebScrapping</h3>

In [0]:
import pandas as pd

# Crear un DataFrame de Pandas con las listas
tabla = {
    'Localidad': loc,
    'Barrio': bar,
    'Hombres': hom,
    'Mujeres' : muj,
    'Total' : tot
}

df = pd.DataFrame(tabla)

# Imprimir el DataFrame
print(df)

                   Localidad  ...    Total
0                      Bronx  ...  114,070
1                      Bronx  ...  142,935
2                      Bronx  ...  121,544
3                      Bronx  ...  129,130
4                      Bronx  ...  163,704
5                      Bronx  ...  130,681
6                      Bronx  ...  136,391
7                      Bronx  ...  142,504
8                      Bronx  ...  187,006
9                      Bronx  ...  159,091
10          Kings (Brooklyn)  ...  152,416
11          Kings (Brooklyn)  ...  131,215
12          Kings (Brooklyn)  ...  136,658
13          Kings (Brooklyn)  ...  123,378
14          Kings (Brooklyn)  ...  111,216
15          Kings (Brooklyn)  ...  126,336
16          Kings (Brooklyn)  ...  120,791
17          Kings (Brooklyn)  ...  156,854
18          Kings (Brooklyn)  ...  198,379
19          Kings (Brooklyn)  ...  139,516
20          Kings (Brooklyn)  ...  110,217
21          Kings (Brooklyn)  ...  151,277
22         

<h3>Creando la sesión de PySpark y transformando el dataframe de pandas en pyspark para mostrar en forma de tabla</h3>

In [0]:
from pyspark.sql import SparkSession #Importamos la libreria de Spark para iniciar sesión y visualizar la tabla

# Crear una SparkSession
spark = SparkSession.builder \
    .appName("Pandas_to_Spark") \
    .getOrCreate()


# Convertir el DataFrame de Pandas a un DataFrame de PySpark
df_spark = spark.createDataFrame(df)

# Visualizar los primeros registros del DataFrame de PySpark
df_spark.display()

Localidad,Barrio,Hombres,Mujeres,Total
Bronx,"Riverdale, Fieldston & Kingsbridge","52,133","61,937","114,070"
Bronx,"Wakefield, Williamsbridge & Woodlawn","65,087","77,848","142,935"
Bronx,"Co-op City, Pelham Bay & Schuylerville","55,615","65,929","121,544"
Bronx,"Pelham Parkway, Morris Park & Laconia","61,233","67,896","129,130"
Bronx,"Belmont, Crotona Park East & East Tremont","75,963","87,740","163,704"
Bronx,"Bedford Park, Fordham North & Norwood","62,664","68,016","130,681"
Bronx,"Morris Heights, Fordham South & Mount Hope","64,748","71,644","136,391"
Bronx,"Concourse, Highbridge & Mount Eden","67,535","74,968","142,504"
Bronx,"Castle Hill, Clason Point & Parkchester","87,605","99,401","187,006"
Bronx,"Hunts Point, Longwood & Melrose","80,447","78,645","159,091"
